In [1]:
# Necessário para exibir as imagens no notebook.
%matplotlib inline

# 1. O Spambase Data Set

O Spambase data set está disponível no site da [UCI](https://archive.ics.uci.edu/ml/datasets/Spambase).O conjunto de dados comtém 4600 mensagens de email classificadas como spam ou não-spam, e o nosso objetivo é construir um classificador que seja capaz de distinguir, com uma boa precisão, um spam de um email normal.

Nessa base de dados, cada email foi convertido em uma sequência de 58 valores numericos (58 atributos), onde:

- [0-47] 48 palavras mais citadas nos emails, onde cada valor representa a probabilidade da palavra ser encontrada no email (*[número de ocorrências]/[total de palavras] x 100 *);
- [48-53] 6 caracteres mais frequentes nos emails, e cada valor representa a probabiidade do caracter ser encontrado no email (*[número de ocorrências]/[total de caracteres] x 100*);
- [54] comprimento médio das palavras escritas em caixa alta;
- [55] comprimento da maior palavra escritas em caixa alta;
- [56] total de letras em maiúsculo no email.
- [57] a classe do email: 1 (spam) ou 0 (não-spam).

\**Para mais informações visite o notebbok da [aula anterior](https://github.com/fernandus16/infnet-aula-1/blob/master/Spambase%20-%20Introdu%C3%A7%C3%A3o.ipynb).*

## O KNN

http://www.inf.ufg.br/mestrado/sites/www.inf.ufg.br.mestrado/files/uploads/Dissertacoes/Fernando%20Chagas.pdf

O método dos k vizinhos mais similares tem sido aplicado na solução de problemas desde o início das pesquisas nessa área e, apesar de simples, tem se mostrado um dos métodos mais eficazes já propostos. Para classificar um documento *d*, ainda não classificado (denominado documento de teste), esse método tradicionalmente realiza as seguintes atividades:
1. A similaridade entre o documento de teste *d* e cada um dos documentos que foram previamente classificados por um especialista (documentos de treino) é calculada utilizando alguma medida de similaridade entre documentos;
2. Os k documentos de treino mais similares ao documento *d* são selecionados.
3. O documento *d* é classificado em determinada categoria de acordo com algum critério de agrupamento dos k vizinhos mais próximos selecionados na etapa anterior (por exemplo, a categoria que possuir a maioria dos k vizinhos mais próximos ao documento de teste *d*).


O critério de similaridade é um aspecto que possui grande influência no desempenho do método kNN. Esse critério é composto pela **medida de similaridade, ou função de distância** (conforme a primeira atividade realizada pelo método kNN), e **pelo critério de seleção** (conforme a segunda atividade realizada pelo método kNN). **O critério de seleção determina a forma de escolha dos k vizinhos de um documento de teste d**. Por exemplo, selecionar os 5 documentos de treino mais similares ao documento de teste d é um critério de seleção


____
## Leitura dos dados

In [2]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split

# Lendo csv
csv_data = pd.read_csv("data/spambase.csv")

# Copiando os dados do csv
data = csv_data.values.copy()

# Embaralhando os dados para garantir aleatoriedade entre as amostras
# np.random.shuffle(data)

# Separando atributos de classes
x = data[:, :-1]  # x tem apenas valores entre a primeira e penúltima coluna
y = data[:, -1]  # y tem os valores da última coluna

# 70% dos dados serão utilizados para treinamento e 30% para o teste
# A divisão será estratificada, serão mantidas as proporções de spam e não spam em cada grupo
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, train_size=0.7, stratify=y)

## Aplicando o K-Means com Sklearn

#### Critério de seleção dos k vizinhos:
 - ‘uniform’: todos os vizinhos tem pesos iguais;
 - ‘distance’: os mais próximos tem maior peso. O vizinho mais próximo tem uma influência maior sobre a decisão.

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score


k = 10
knn_clf = KNeighborsClassifier(k, weights='distance')
# Treinamento
knn_clf.fit(x_treino, y_treino)
# Predição
y_knn_pred = knn_clf.predict(x_teste)

# precision_score = tp / (tp + fp) onde tp ee o numero verdadeiro positivo fp, falsos positivos.
print u"Precisão do KNN: ", precision_score(y_knn_pred, y_teste, average='binary')

Precisão do KNN:  0.748161764706


## Fazendo um comparativos com os métodos apresentados na aula anterior


Primeiro, uma função para executar e avaliar os métodos

In [4]:
from datetime import datetime

def get_clf_precision(clf):

    start_time = datetime.now()
    clf.fit(x_treino, y_treino)
    fit_time = datetime.now() - start_time
    print "Fit time: ", fit_time.microseconds
    
    # Predição
    y_pred = clf.predict(x_teste)
    pred_time = datetime.now() - start_time - fit_time
    print "Pred time: ", pred_time.microseconds
    
    return precision_score(y_pred, y_teste, average='binary')

Executando:
 - KNN;
 - Bernoulli Naive Bayes;
 - Multinomial Naive Bayes;
 - Árvore de Decisão.

In [5]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

clf_list = {
    u"KNN": KNeighborsClassifier(k, weights='distance'),
    u"Bernoulli Naive Bayes": BernoulliNB(),
    u"Multinomial Naive Bayes": MultinomialNB(),
    u"Árvore de Decisão": DecisionTreeClassifier(criterion="gini", random_state=9, max_depth=9),
}

for clf_name, clf in clf_list.iteritems():
    print u"Classificador %s - precisão de %s" % (clf_name, get_clf_precision(clf))

Fit time:  9158
Pred time:  57708
Classificador KNN - precisão de 0.748161764706
Fit time:  56385
Pred time:  1873
Classificador Bernoulli Naive Bayes - precisão de 0.819852941176
Fit time:  39895
Pred time:  1086
Classificador Árvore de Decisão - precisão de 0.845588235294
Fit time:  2022
Pred time:  328
Classificador Multinomial Naive Bayes - precisão de 0.672794117647
